In [ ]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

In [ ]:
import os 
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBase
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from src.architectures.adv.guendel19 import densenet
from src.metrics.metrics import F2Score

In [ ]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


In [ ]:
!remote_access/get_tunnels.sh

In [ ]:
metrics = [tf.keras.metrics.AUC(multi_label=True),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           F2Score(),
           tf.keras.metrics.BinaryAccuracy()
          ]

In [ ]:

chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  epochs=10,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=8,
                                  metrics = metrics)
chexpert_benchmark.as_dict()

In [ ]:
chestxray14_columns = ['Edema', 'Atelectasis', 'Pneumonia',
                       'Pleural_Thickening', 'Cardiomegaly', 'Infiltration', 'Consolidation',
                       'Fibrosis', 'No Finding', 'Effusion', 'Nodule', 'Mass', 'Hernia',
                       'Emphysema', 'Pneumothorax']

chestxray14_benchmark = Benchmark(Path(os.environ.get("CHESTXRAY14_DATASET_DIRECTORY")),
                                  chestxray14_columns,
                                  epochs=10,
                                  train_labels="meta/data/labels.csv",
                                  path_column="Image Index",
                                  path_column_prefix="images/",
                                  split_group='Patient ID',
                                  batch_size=8,
                                  metrics=metrics)
chestxray14_benchmark.as_dict()

In [ ]:
model_resnet_chestxray14 = SimpleBaseArchitecture(ResNet152V2, len(chestxray14_columns))
model_resnet_chexpert = SimpleBaseArchitecture(ResNet152V2, len(chexpert_columns))

model_densenet_chestxray14 = SimpleBaseArchitecture(DenseNet121, len(chestxray14_columns))
model_densenet_chexpert = SimpleBaseArchitecture(DenseNet121, len(chexpert_columns))

model_inceptionnet_chestxray14 = SimpleBaseArchitecture(InceptionV3, len(chestxray14_columns))
model_inceptionnet_chexpert = SimpleBaseArchitecture(InceptionV3, len(chexpert_columns))

In [ ]:
experiment_resnet_chexpert = Experiment(chexpert_benchmark, model_resnet_chexpert, "ResNet152V2_CheXpert_8")
experiment_densenet_chexpert = Experiment(chexpert_benchmark, model_densenet_chexpert, "DenseNet121_CheXpert_32")
experiment_inceptionnet_chexpert = Experiment(chexpert_benchmark, model_inceptionnet_chexpert, "InceptionV3_CheXpert_32")

In [ ]:
experiment_resnet_chestxray14 = Experiment(chestxray14_benchmark, model_resnet_chestxray14, "ResNet152V2_ChestXray14_32")
experiment_densenet_chestxray14 = Experiment(chestxray14_benchmark, model_densenet_chestxray14, "DenseNet121_ChestXray14_32")
experiment_inceptionnet_chestxray14 = Experiment(chestxray14_benchmark, model_inceptionnet_chestxray14, "InceptionV3_ChestXray14_32")

In [ ]:
experiment_resnet_chexpert_result =  experiment_resnet_chexpert.run()

In [ ]:
experiment_densenet_chexpert_result =  experiment_densenet_chexpert.run()

In [ ]:
experiment_inceptionnet_chexpert_result =  experiment_inceptionnet_chexpert.run()

In [ ]:
experiment_resnet_chestxray14_result =  experiment_resnet_chestxray14.run()

In [ ]:
experiment_densenet_chestxray14_result =  experiment_densenet_chestxray14.run()

In [ ]:
experiment_inceptionnet_chestxray14_result =  experiment_inceptionnet_chestxray14.run()

In [ ]:
predictions = experiment_inceptionnet_chexpert.model.predict(
            experiment_inceptionnet_chexpert.benchmark.testgen, steps=len(experiment_inceptionnet_chexpert.benchmark.testgen), verbose=1)

In [ ]:
true_labels = experiment_inceptionnet_chexpert.benchmark.testgen.get_encoded_labels()

In [ ]:
model_guendel_chestxray14 = densenet(classes=len(chestxray14_columns))
model_guendel_chexpert = densenet(classes=len(chexpert_columns))

experiment_guendel_chestxray14 = Experiment(chestxray14_benchmark, model_guendel_chestxray14, "Guendel19_ChestXray14")
experiment_guendel_chexpert = Experiment(chexpert_benchmark, model_guendel_chexpert, "Guendel19_CheXpert")



In [ ]:
experiment_guendel_chestxray14_result =  experiment_guendel_chestxray14.run()

In [ ]:
experiment_guendel_chexpert_result =  experiment_guendel_chexpert.run()